In [111]:
# Source: https://www.tensorflow.org/tutorials/quickstart/beginner

In [112]:
%load_ext tensorboard
import tensorflow as tf 
import datetime
import time

# Clear any logs from previous runs
! rm -rf ./logs/


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [113]:
# Import the mnist dataset

In [114]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [115]:
# Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [116]:
# For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.


predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.67502666,  0.73401946,  0.2616675 , -0.61586   ,  0.49298626,
         0.20569378, -0.46536547,  0.7029633 ,  0.13632685, -0.35649505]],
      dtype=float32)

In [117]:
# The tf.nn.softmax function converts these logits to "probabilities" for each class:
tf.nn.softmax(predictions).numpy()

# The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [118]:
# This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.
# This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.


In [119]:
loss_fn(y_train[:1], predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [120]:
# Log stats

In [121]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [122]:
# The Model.fit method adjusts the model parameters to minimize the loss:

In [131]:
# tf.profiler.experimental.start('logdir')
# Train the model here
tic = time.clock()

model.fit(x_train, y_train, epochs=5,callbacks=[tensorboard_callback])

toc = time.clock()

# tf.profiler.experimental.stop()

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0652 - accuracy: 0.9790
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0581 - accuracy: 0.9814: 0s
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0517 - accuracy: 0.9833
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0465 - accuracy: 0.9844
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0432 - accuracy: 0.9853


In [124]:
print(float("{0:.2f}".format(toc-tic)),"seconds")

33.86 seconds


In [125]:
# The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [126]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0870 - accuracy: 0.9737


[0.08699800074100494, 0.9736999869346619]

In [127]:
# The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

# If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [128]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [129]:
probability_model(x_test[:5])


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.4603664e-08, 5.8867639e-10, 3.4321088e-06, 4.1967203e-05,
        3.6607245e-12, 3.1187793e-07, 5.2247872e-12, 9.9994731e-01,
        1.8689356e-07, 6.7084929e-06],
       [5.6127383e-09, 1.5357346e-05, 9.9998379e-01, 8.7615700e-07,
        3.8757056e-15, 3.1008489e-09, 1.2924369e-08, 1.3505268e-14,
        1.9906135e-08, 1.7546664e-13],
       [3.5821545e-06, 9.9796778e-01, 1.2765345e-04, 2.1615151e-05,
        2.8039937e-04, 5.2750340e-05, 4.0016839e-05, 1.3735784e-03,
        1.3116187e-04, 1.4678481e-06],
       [9.9994779e-01, 8.4205587e-10, 2.1441263e-05, 1.7454579e-07,
        4.3389807e-07, 4.6268977e-07, 8.6404543e-06, 2.4642393e-06,
        2.2708899e-09, 1.8564007e-05],
       [4.2639692e-07, 1.3753933e-10, 1.8468718e-07, 8.2872864e-09,
        9.9376923e-01, 2.1730143e-08, 2.2553579e-06, 5.9943581e-05,
        4.6400026e-09, 6.1678616e-03]], dtype=float32)>

In [130]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 9809), started 2:54:27 ago. (Use '!kill 9809' to kill it.)